# Exercise08 : Publish as a Web Service

Finally we publish our model as a web service.

Before running this code, complete the model training in "[Exercise04 : Train on Remote GPU Virtual Machine](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise04_train_remote.ipynb)".

*back to [index](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/)*

## Get workspace settings

Before starting, you must read your configuration settings. (See "[Exercise01 : Prepare Config Settings](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise01_prepare_config.ipynb)".)

In [1]:
from azureml.core import Workspace
import azureml.core

ws = Workspace.from_config()

## Get the registered model

Get the trained model by MNIST dataset.<br>
**Before running this code, complete the model training in "[Exercise04 : Train on Remote GPU Virtual Machine](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise04_train_remote.ipynb)".**

In [2]:
from azureml.core.model import Model

registered_model = Model(ws, 'mnist_model_test')

## Deploy as web service

In order to deploy as web service, first we generate the following scoring source.    
This entry script in AML should include both ```init()``` and ```run()```.

In [3]:
%%writefile score.py
import json
import tensorflow as tf
from azureml.core.model import Model

def init():
    global pred_fn
    model_path = Model.get_model_path(model_name='mnist_model_test')
    pred_fn = tf.contrib.predictor.from_saved_model(model_path)

def run(raw_data):
    try:
       data = json.loads(raw_data)['data']
       result = pred_fn({'inputs': data})
       return result['classes'].tolist()
    except Exception as e:
       result = str(e)
       return 'Internal Exception : ' + result

Writing score.py


Create deploy configuration for preparation.<br>
In this tutorial, we will deploy our serving in Azure Container Instance (ACI) with anonymous access permission.

In [4]:
from azureml.core.webservice import AciWebservice, Webservice

aci_conf = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1, 
    description='This is a tensorflow example.')

Next, create inference configuration for preparation.

In [5]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

# Generate conda dependency
conda_dependency = CondaDependencies.create()
conda_dependency.add_pip_package('tensorflow==1.15')
### Or you can also write as follows (make sure to insert 'azureml-defaults' module)
#conda_dependency = CondaDependencies.create(pip_packages=['azureml-defaults', 'tensorflow==1.15'])

# Create environment and set the previous conda dependency
myenv = Environment(name="myenv")
myenv.python.conda_dependencies = conda_dependency

# Create inference config with score.py
inf_conf = InferenceConfig(
    entry_script="score.py",
    environment=myenv)

Now, deploy as a web service !

In [6]:
svc = Model.deploy(
    name='my-mnist-service',
    deployment_config=aci_conf,
    models=[registered_model],
    inference_config=inf_conf,
    workspace=ws)
svc.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-23 08:14:40+00:00 Creating Container Registry if not exists.
2021-08-23 08:14:40+00:00 Registering the environment.
2021-08-23 08:14:43+00:00 Building image..
2021-08-23 08:22:42+00:00 Generating deployment configuration.
2021-08-23 08:22:44+00:00 Submitting deployment to compute..
2021-08-23 08:22:48+00:00 Checking the status of deployment my-mnist-service..
2021-08-23 08:25:41+00:00 Checking the status of inference endpoint my-mnist-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [7]:
# See details, if error has occured
print(svc.get_logs())

2021-08-23T08:25:17,379725500+00:00 - iot-server/run 
2021-08-23T08:25:17,386434100+00:00 - rsyslog/run 
2021-08-23T08:25:17,395805700+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-23T08:25:17,398135200+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-23T08:25:17,764669100+00:00 - iot-server/finish 1 0
2021-08-23T08:25:17,770766800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (65)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 93
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-23 08:25:23,364 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-23 08:25:23,364 | root | INFO | Starting up request id generator
2021-08-23 08:25:23,364 | root | INFO | Star

Check service url

In [8]:
svc.scoring_uri

'http://1582fdf0-bd82-4d2f-8212-39a25a810459.eastus.azurecontainer.io/score'

## Test your web service

Let's invoke your web service and check the returned results in Python.

In [9]:
import requests
import json

import tensorflow as tf

# Read data by tensor
tfdata = tf.data.TFRecordDataset('./data/test.tfrecords')
iterator = tf.compat.v1.data.make_one_shot_iterator(tfdata)
data_org = iterator.get_next()
###
data_exam = tf.parse_single_example(
    data_org,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    })
data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
data_image.set_shape([784])
data_image = tf.cast(data_image, tf.float32) * (1. / 255)
data_label = tf.cast(data_exam['label'], tf.int32)

# Run tensor and generate data
with tf.Session() as sess:
    image_arr = []
    label_arr = []
    for i in range(3):
        image, label = sess.run([data_image, data_label])
        image_arr.append(image.tolist())
        label_arr.append(label)

# Invoke web service !
headers = {'Content-Type':'application/json'}
# for AKS deployment (in production), provide service key in the header as below :
# api_key1, api_key2 = svc.get_keys()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key1)} 
values = json.dumps(image_arr)
input_data = "{\"data\": " + values + "}"
http_res = requests.post(
    svc.scoring_uri,
    input_data,
    headers = headers)
print('Predicted : ', http_res.text)
print('Actual    : ', label_arr)

Predicted :  [7, 2, 1]
Actual    :  [7, 2, 1]


## Remove service

In [10]:
svc.delete()